## METROPOLIS HASTINGS

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, gamma, poisson

# Load the data
data = np.loadtxt('premier_league_2013_2014.dat')
y_g1 = data[:, 0]  # Goals scored by home team
y_g2 = data[:, 1]  # Goals scored by away team
h_g = data[:, 2].astype(int)  # Home team index
a_g = data[:, 3].astype(int)  # Away team index

# Parameters
n_teams = 20
n_games = len(data)
tau_0 = 0.0001  # Prior precision for home
tau_1 = 0.0001  # Prior precision for mean parameters
alpha = 0.1     # Gamma shape parameter
beta = 0.1      # Gamma rate parameter

# Set up parameter vector
# Order: home, att_1,...,att_19, def_1,...,def_19, mu_att, mu_def, tau_att, tau_def
n_params = 1 + (n_teams-1) + (n_teams-1) + 4  # home + att + def + hyperparams
att_start_idx = 1
def_start_idx = att_start_idx + (n_teams-1)
hyper_start_idx = def_start_idx + (n_teams-1)

def log_likelihood(params):
    """Compute log likelihood of the data given parameters."""
    home = params[0]
    att = np.zeros(n_teams)
    att[1:] = params[att_start_idx:def_start_idx]  # att_0 = 0
    
    def_ = np.zeros(n_teams)
    def_[1:] = params[def_start_idx:hyper_start_idx]  # def_0 = 0
    
    mu_att = params[hyper_start_idx]
    mu_def = params[hyper_start_idx + 1]
    tau_att = params[hyper_start_idx + 2]
    tau_def = params[hyper_start_idx + 3]
    
    # Ensure tau parameters are positive
    if tau_att <= 0 or tau_def <= 0:
        return -np.inf
    
    # Calculate theta values
    log_theta_g1 = home + att[h_g] - def_[a_g]
    log_theta_g2 = att[a_g] - def_[h_g]
    
    theta_g1 = np.exp(log_theta_g1)
    theta_g2 = np.exp(log_theta_g2)
    
    # Log likelihood for observed goals
    ll_goals = np.sum(poisson.logpmf(y_g1, theta_g1) + poisson.logpmf(y_g2, theta_g2))
    
    # Log prior for home
    ll_home = norm.logpdf(home, 0, np.sqrt(1/tau_0))
    
    # Log prior for attack parameters
    ll_att = np.sum(norm.logpdf(att[1:], mu_att, np.sqrt(1/tau_att)))
    
    # Log prior for defense parameters
    ll_def = np.sum(norm.logpdf(def_[1:], mu_def, np.sqrt(1/tau_def)))
    
    # Log hyperpriors
    ll_mu_att = norm.logpdf(mu_att, 0, np.sqrt(1/tau_1))
    ll_mu_def = norm.logpdf(mu_def, 0, np.sqrt(1/tau_1))
    ll_tau_att = gamma.logpdf(tau_att, alpha, scale=1/beta)
    ll_tau_def = gamma.logpdf(tau_def, alpha, scale=1/beta)
    
    return ll_goals + ll_home + ll_att + ll_def + ll_mu_att + ll_mu_def + ll_tau_att + ll_tau_def

def metropolis_hastings(n_burn, n_samples, thin, sigma):
    """
    Run Metropolis-Hastings algorithm.
    
    Args:
        n_burn: Number of burn-in samples
        n_samples: Number of samples to collect after burn-in
        thin: Thinning factor (keep every thin-th sample)
        sigma: Standard deviation for isotropic Gaussian proposal
    
    Returns:
        burn_samples: Samples from burn-in phase
        samples: Collected samples after burn-in
    """
    # Initialize parameters at 0
    current_params = np.zeros(n_params)
    current_params[hyper_start_idx + 2] = 1.0  # tau_att init
    current_params[hyper_start_idx + 3] = 1.0  # tau_def init
    
    current_ll = log_likelihood(current_params)
    
    # Store burn-in samples
    burn_samples = np.zeros((n_burn, n_params))
    
    # Burn-in phase
    for i in range(n_burn):
        # Propose new parameters
        proposal = current_params + np.random.normal(0, sigma, n_params)
        
        # Handle tau parameters to ensure they're positive
        if proposal[hyper_start_idx + 2] <= 0:
            proposal[hyper_start_idx + 2] = np.abs(proposal[hyper_start_idx + 2])
        if proposal[hyper_start_idx + 3] <= 0:
            proposal[hyper_start_idx + 3] = np.abs(proposal[hyper_start_idx + 3])
        
        # Compute log likelihood of proposal
        proposal_ll = log_likelihood(proposal)
        
        # Accept/reject step
        log_accept_ratio = proposal_ll - current_ll
        
        if np.log(np.random.random()) < log_accept_ratio:
            current_params = proposal
            current_ll = proposal_ll
        
        burn_samples[i] = current_params
        
        if (i+1) % 1000 == 0:
            print(f"Burn-in iteration {i+1}/{n_burn}")
    
    # Collect samples with thinning
    total_iter = n_samples * thin
    samples = np.zeros((n_samples, n_params))
    
    for i in range(total_iter):
        # Propose new parameters
        proposal = current_params + np.random.normal(0, sigma, n_params)
        
        # Handle tau parameters to ensure they're positive
        if proposal[hyper_start_idx + 2] <= 0:
            proposal[hyper_start_idx + 2] = np.abs(proposal[hyper_start_idx + 2])
        if proposal[hyper_start_idx + 3] <= 0:
            proposal[hyper_start_idx + 3] = np.abs(proposal[hyper_start_idx + 3])
        
        # Compute log likelihood of proposal
        proposal_ll = log_likelihood(proposal)
        
        # Accept/reject step
        log_accept_ratio = proposal_ll - current_ll
        
        if np.log(np.random.random()) < log_accept_ratio:
            current_params = proposal
            current_ll = proposal_ll
        
        # Store sample if it's a thinning iteration
        if (i+1) % thin == 0:
            samples[(i+1)//thin - 1] = current_params
        
        if (i+1) % 1000 == 0:
            print(f"Sampling iteration {i+1}/{total_iter}")
    
    return burn_samples, samples

# Run MCMC
n_burn = 5000
n_samples = 5000
thin = 5
sigma = 0.05

burn_samples, samples = metropolis_hastings(n_burn, n_samples, thin, sigma)

# Plot trace plot for home parameter (first parameter)
plt.figure(figsize=(12, 6))

# Burning phase
plt.subplot(1, 2, 1)
plt.plot(range(n_burn), burn_samples[:, 0])
plt.title('Trace plot for home (burn-in phase)')
plt.xlabel('Iteration')
plt.ylabel('home')

# Post burn-in samples
plt.subplot(1, 2, 2)
plt.plot(range(n_samples), samples[:, 0])
plt.title('Trace plot for home (post burn-in samples)')
plt.xlabel('Sample')
plt.ylabel('home')

plt.tight_layout()
plt.savefig('home_trace.png')
plt.show()

# Print mean estimate of home
print(f"Estimated posterior mean of home: {np.mean(samples[:, 0]):.4f}")

FileNotFoundError: premier_league_2013_2014.dat not found.